# **Elements of Machine Learning 2024** <font size=4 color='gray'>Alan Reyes-Figueroa</font>
#### <font color='gray'>Imputación de Datos</font>

# Introduction

In this tutorial we will explore some basic techniques of data imputation, that is, how to fill missing values in a dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer

In [ ]:
#pd.set_option('max_rows', 30)

In [ ]:
data = pd.read_csv('hpi-data-2016.csv')

In [ ]:
data.head(10)

In [ ]:
data.shape

# Missing data

Entries missing values are given the value `NaN`, short for "Not a Number". For technical reasons these `NaN` values are always of the `float64` dtype.

Pandas provides some methods specific to missing data. To select `NaN` entries you can use `pd.isnull()` (or its companion `pd.notnull()`). This is meant to be used thusly:

In [ ]:
data.isnull().sum()

Replacing missing values is a common operation.  Pandas provides a really handy method for this problem: `fillna()`. `fillna()` provides a few different strategies for mitigating such data. For example, we can simply replace each `NaN` with an `"Unknown"`:

In [ ]:
data['GINI-index'].fillna("Unknown", inplace=True)

In [ ]:
data.head(15)

## Fill with zeros

In [ ]:
## reemplazar con 0
data = pd.read_csv('hpi-data-2016.csv')
data2 = data.copy()

In [ ]:
data2['GINI-index'] = data2['GINI-index'].fillna(0)

In [ ]:
data2.head()

In [ ]:
data['GINI-index'].mean()

In [ ]:
data2['GINI-index'].mean()

In [ ]:
data['GINI-index'].hist()

In [ ]:
data2['GINI-index'].hist()

## Fill with statistical resumes

In [ ]:
## reemplazar con la media

data3 = data.copy()
media = data3['GINI-index'].mean()
print(media)

In [ ]:
data3['GINI-index'] = data3['GINI-index'].fillna(media)

In [ ]:
data3.head()

In [ ]:
data3['GINI-index'].mean()

In [ ]:
data['GINI-index'].hist()      # pandas histogram descarta los faltantes

In [ ]:
data3['GINI-index'].hist()

In [ ]:
# reemplazar con la mediana

mediana = data['GINI-index'].median()
print(mediana)
data3['GINI-index'] = data3['GINI-index'].fillna(mediana)

In [ ]:
moda = data['Region'].mode()
print(moda)

In [ ]:
moda = data['GINI-index'].mode()
print(moda)

In [ ]:
data['GINI-index'].value_counts()

## Fill with group or cluster mean 

Sometimes it is more convenient to replace missing values with group means, where the groups correspond to some categorial variable values, or some clustering scheme.

In [ ]:
regions = data.Region.unique()
regions

In [ ]:
data.groupby('Region')['GINI-index'].mean()
#means = data.groupby('Region')['GINI-index'].mean().values
#means

In [ ]:
means = data.groupby('Region')['GINI-index'].mean().values
means

In [ ]:
data4 = data.copy()
data4['GINI-index'] = data4['GINI-index'].fillna(data3.groupby('Region')['GINI-index'].transform('mean'))
data4

In [ ]:
data4['GINI-index'].hist()

## Fill with linear regression estimations

In [ ]:
data.corr()

In [ ]:
plt.figure()
sns.pairplot(data[['Inequality-of-Outcomes', 'GINI-index']])
plt.show()

In [ ]:
datadropna = data.dropna()

In [ ]:
datadropna

In [ ]:
x = datadropna['Inequality-of-Outcomes'].values
y = datadropna['GINI-index'].values

In [ ]:
x

In [ ]:
y

In [ ]:
x.shape

In [ ]:
n = x.shape[0]
n

In [ ]:
# construct design matrix

X = np.stack([np.ones(n), x]).T
X.shape

In [ ]:
# compute linear regression coefficients

c = np.linalg.inv(X.T @ X) @ (X.T) @ y

In [ ]:
c

In [ ]:
def regresion(x, c):
    yhat = c @ np.hstack([np.array([1.]), x])
    return yhat

In [ ]:
data4 = data.copy()

In [ ]:
data4

In [ ]:
for i in range(0, data4.shape[0]):
    if np.isnan(data4['GINI-index'][i]):
        y = regresion(data4['Inequality-of-Outcomes'][i], c)
        data4['GINI-index'][i] = y

In [ ]:
data4

In [ ]:
data['GINI-index'].hist()

In [ ]:
data4['GINI-index'].hist()

## K nearest neighbour imputation

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
# define imputer
imputer = KNNImputer(n_neighbors=10, weights='uniform', metric='nan_euclidean')

In [ ]:
X = data.iloc[:,2:].values

# fit
imputer.fit(X)

In [ ]:
# transform the dataset
Xtrans = imputer.transform(X)

In [ ]:
data4 = data.copy()
data4['GINI-index'] = imputer.transform(X)[:,-1]

In [ ]:
data4

In [ ]:
data4['GINI-index'].hist(bins=35)

In [ ]:
data['GINI-index'].hist(bins=35)